In [58]:
import torch
import evaluate
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer

In [ ]:
!wget https://raw.githubusercontent.com/rsuh-python/mag2023NN/refs/heads/main/07-transformers/train.txt
!wget https://raw.githubusercontent.com/rsuh-python/mag2023NN/refs/heads/main/07-transformers/val.txt

Наш кастомный датасет - это просто текстоый файл, где каждое предложение отделено от другого пустой строкой, а лейблы записаны через таб к токенам. Нам нужно сперва распарсить этот датасет, потом написать класс для него и наконец собрать все в dataloader.

In [59]:
def parse_dataset(filepath):
    sentences, labels = [], []
    with open(filepath, 'r', encoding='utf-8') as file:
        current_sentence, current_labels = [], []
        for line in file:
            line = line.strip()
            if not line:  # конец предложения
                if current_sentence:
                    sentences.append(current_sentence)
                    labels.append(current_labels)
                current_sentence, current_labels = [], []
            else:
                token, tag = line.split('\t')
                current_sentence.append(token)
                current_labels.append(tag)
        # добавим последнее предложение на случай, если файл не заканчивался на пустую строчку
        if current_sentence:
            sentences.append(current_sentence)
            labels.append(current_labels)
    return sentences, labels

In [60]:
class NERDataset(Dataset):
    def __init__(self, sentences, labels, tokenizer, label2id, max_length=128):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.label2id = label2id
        self.max_length = max_length

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        tokens = self.sentences[idx]
        tags = self.labels[idx]

        # токенизируем
        encoding = self.tokenizer(
            tokens,
            is_split_into_words=True,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_offsets_mapping=True
        )
        
        # свяжем лейблы с токенизированным аутпутом: у нас subwords
        labels = []
        word_ids = encoding.word_ids()  # мапим ID токенов с ID слов
        previous_word_id = None

        for word_id in word_ids:
            if word_id is None:  # Special tokens or padding
                labels.append(-100)
            elif word_id != previous_word_id:  # начало нового слова
                labels.append(self.label2id.get(tags[word_id], -100))
            else:  # Subword
                labels.append(-100)
            previous_word_id = word_id

        encoding["labels"] = labels
        encoding.pop("offset_mapping")  # для модели не нужно

        return {key: torch.tensor(val) for key, val in encoding.items()}

In [61]:
def create_dataloader(path, tokenizer, label2id, batch_size=16, max_length=128):
    sentences, labels = parse_dataset(path)
    dataset = NERDataset(sentences, labels, tokenizer, label2id, max_length=max_length)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:
# определим теги и их ID
labels = ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "B-MISC", "I-MISC"]
label2id = {label: idx for idx, label in enumerate(labels)}
id2label = {idx: label for label, idx in label2id.items()}

# загрузим токенизатор и модель
model_checkpoint = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# соберем лоудеры
batch_size = 32
train_dataloader = create_dataloader('train.txt', tokenizer, label2id, batch_size=batch_size)
val_dataloader = create_dataloader('val.txt', tokenizer, label2id, batch_size=batch_size)

# проверим, что все ок
batch = next(iter(train_dataloader))

Посмотрим, что у нас в батче

In [ ]:
print(batch['input_ids'][0], batch['labels'][0])
tokenizer.decode(batch['input_ids'][0])

In [ ]:
# приготовим модель к обучению
# transformers должен предупредить, что нам надо бы потренировать модельку - значит, все ок.
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

In [64]:
# будем использовать стандартную метрику seqeval из модуля evaluate
metric = evaluate.load('seqeval')

def compute_metrics(p):
    predictions, labels = p

    # логиты в индексы
    predictions = predictions.argmax(axis=-1)

    # союда будем собирать необходимое барахло
    true_predictions = []
    true_labels = []

    for prediction, label in zip(predictions, labels):
        # Align predictions and labels, ignoring special tokens (-100)
        current_predictions = []
        current_labels = []

        for pred, lab in zip(prediction, label):
            if lab != -100:  # игнорим паддинг
                current_predictions.append(id2label[pred])
                current_labels.append(id2label[lab])

        # добавляем в главные два листа
        true_predictions.append(current_predictions)
        true_labels.append(current_labels)

    # пихнем в метрику и получим результат
    results = metric.compute(predictions=true_predictions, references=true_labels)

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


В трансформерах все петли написаны за нас: нам остается передать аргументы для обучения (их много) и собственно запустить трейнер.

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",          # Output directory
    eval_strategy="epoch",    # Evaluate after each epoch
    learning_rate=5e-5,             # Learning rate
    per_device_train_batch_size=32, # Batch size for training
    per_device_eval_batch_size=32,  # Batch size for evaluation
    num_train_epochs=3,             # Number of epochs
    weight_decay=0.01,              # Strength of weight decay
    logging_dir="./logs",           # Directory for storing logs
    logging_steps=10,               # Log every 10 steps
    save_strategy="epoch",          # Save model after each epoch
    load_best_model_at_end=True,    # Load the best model after training
    metric_for_best_model="f1",     # Use F1 score to choose the best model
)

In [67]:
# инициализировали
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataloader.dataset,  # Training dataset
    eval_dataset=val_dataloader.dataset,   # Evaluation dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
# собственно обучение - автоматически делает логи
trainer.train()

# оценим модельку
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")

# Сохраним, что получилось
trainer.save_model("./ner_model")


Сделаем, так сказать, качественную оценку:

In [ ]:
def predict(sentence, tokenizer, model):
    # токенизируем исходное предложение
    tokens = tokenizer(
        sentence.split(),
        is_split_into_words=True,
        return_tensors="pt",
        truncation=True
    )
    
    word_ids = tokens.word_ids()  # мапим токены по индексам слов
    with torch.no_grad():
        tokens.to('cuda')
        outputs = model(**tokens)
        predictions = outputs.logits.argmax(dim=-1).squeeze().tolist()

    # будем элайнить лейблы по словам
    aligned_predictions = []
    current_word_id = None

    for word_id, prediction in zip(word_ids, predictions):
        if word_id is not None and word_id != current_word_id:  # начало нового слова
            aligned_predictions.append(id2label[prediction])
            current_word_id = word_id

    # зазипим результаты
    result = list(zip(sentence.split(), aligned_predictions))
    return result


# Проверка
example_sentence = "Eftir að hafa gegnt herskyldu í fyrri heimsstyrjöldinni hóf Hubble störf við stjörnuathugunarstöðina á Wilson - fjalli í Kaliforníu."
print(predict(example_sentence, tokenizer, model))


Использование готовых инструментов - хорошо, но иногда перед нами стоит задача модифицировать архитектуру модели, а то и вообще написать свою собственную с нуля, только используя эмбеддинги берта. Давайте перепишем архитектуру модели без использования автомодели трансформеров.

In [ ]:
import torch.nn as nn
from transformers import BertModel

class Model(nn.Module):
    def __init__(self, num_of_classes):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-multilingual-cased")
        self.classifier = nn.Linear(768, num_of_classes)

    def forward(self, input_ids, attention_mask, token_type_ids=None, labels=None):
        # получим эмбеддинги токенов от берта
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids  
        )
        sequence_output = outputs.last_hidden_state  # Shape: (batch_size, seq_len, hidden_size)

        # собственно классификатор
        logits = self.classifier(sequence_output)  # Shape: (batch_size, seq_len, num_classes)

        # Чтобы использовать нашу модель с трейнером трансформеров, нам нужно тут же и лосс посчитать
        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            # Flatten logits and labels for loss computation
            logits_flat = logits.view(-1, logits.shape[-1])  # Shape: (batch_size * seq_len, num_classes)
            labels_flat = labels.view(-1)  # Shape: (batch_size * seq_len)
            loss = loss_fn(logits_flat, labels_flat)  # Scalar loss

        return (loss, logits) if loss is not None else logits


Удостоверимся, что наша модель адекватно работает с датасетом. Батч возвращает нам словарь с ключами, который при распаковке как раз даст нам все то, что мы прописали в форварде:

In [ ]:
model = Model(len(labels))
model(**batch)

In [ ]:
# инициализируем повторно трейнер, но уже с новой самописной моделькой
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataloader.dataset,  # Training dataset
    eval_dataset=val_dataloader.dataset,   # Evaluation dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")
trainer.save_model("./ner_model")

Ну и проверим точно так же, как предыдущую версию:

In [ ]:
example_sentence = "Eftir að hafa gegnt herskyldu í fyrri heimsstyrjöldinni hóf Hubble störf við stjörnuathugunarstöðina á Wilson - fjalli í Kaliforníu."
print(predict(example_sentence, tokenizer, model))

Задание. 

В текущей версии задачи мы берем, по сути, только предсказание модели для первого подслова в слове: но что, если остальные подслова могли бы тоже влиять? Попробуйте доработать код таким образом, чтобы в обучающем датасете каждому подслову слова приписывался тег всего слова, а при предсказании модель выбирала тег слова более обдуманно: например, при трех и более подсловах голосованием. Поэкспериментируйте!